In [1]:
# xmlrpc_server.ipynb

from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler
import xmlrpc.client
import datetime
import pandas as pd
import pickle

class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 8008), 
                            requestHandler=RequestHandler)

#server.register_introspection_functions()

#stats_server = xmlrpc.client.ServerProxy("http://localhost:8018")
stats_server = xmlrpc.client.ServerProxy("http://localhost:7000")

def get_event_logs(operation_type, start_time, end_time):
    stat = stats_server.get_event_log(operation_type, start_time, end_time)
    return stat

server.register_function(get_event_logs, 'get_event_logs')

# Тест
def ping():
    return True
server.register_function(ping, 'ping')

# Время сервера
def now():
    return datetime.datetime.now()
server.register_function(now, 'now')

# Отображение строкового вида, типа и значений
def show_type(arg):
    return (str(arg), str(type(arg)), arg)
server.register_function(show_type, 'type')

# Сумма
def test_sum(a, b):
    return a + b
server.register_function(test_sum, 'sum')

# Степень
def test_pow(a, b):
    
    return a**b
server.register_function(test_pow, 'pow')

# Проверка нахождения клиента в черном списке c использованием Pandas Data Frame
def black_list_check(sname):
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    exist = any(frame['Surname'] == sname)
    if (exist == True):
        return sname + ": "+ "bad_boy"
    else:
        return sname + ": "+ "good_boy"

server.register_function(black_list_check, 'black_list_check')

# Проверка нахождения клиента в черном списке c использованием Pandas Data Frame
def black_list_check_par(sname, name, patronym, date):
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    result = frame.query('Surname == @sname and Name == @name and Patronym == @patronim and Birth == @date')
    if (result.empty):
        return f'{sname} {name} {patronim}, {date} : good boy'
    else:
        return f'{sname} {name} {patronim}, {date} : bad boy'

server.register_function(black_list_check_par, 'black_list_check_par')

# Бинарная передача данных
def send_back_binary(bin_data):
    data = bin_data.data
    return xmlrpc.client.Binary(data)
server.register_function(send_back_binary, 'send_back_binary')

# бинаризация с параметром
def send_back_binary_par(bin_data, threshold):
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]
    
    if len(img_arr.shape) == 3:
        for i in range(height):
            for j in range(width):
                res = 1
                for k in range(3):
                    res *= (img_arr[i][j][k] > threshold[k]) * 255
                    
                res = 255 if res >= 255 else 0
                
                for k in range(3):
                    img_arr[i][j][k] = res
    elif len(img_arr.shape) == 2:
        for i in range(height):
            for j in range(width):
                img_arr[i][j] = (img_arr[i][j] > threshold) * 255
    
    pimg = pickle.dumps(img_arr)
    return xmlrpc.client.Binary(pimg)
server.register_function(send_back_binary_par, 'send_back_binary_par')

# Инверсия цвета
# На вход изображение RGB размерности (M, N, 3) со значениями 0-255
def send_back_inversion(bin_data):
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]   
    
    if len(img_arr.shape) == 3:
        for i in range(height):
            for j in range(width):
                for k in range(3):
                    img_arr[i][j][k] = 255 - img_arr[i][j][k]
    elif len(img_arr.shape) == 2:
        for i in range(height):
            for j in range(width):
                img_arr[i][j] = 255 - img_arr[i][j]
    pimg = pickle.dumps(img_arr)        
    return xmlrpc.client.Binary(pimg)
server.register_function(send_back_inversion, 'color_inversion')

# перевенуть картинку в вертикали
def send_back_flipped(bin_data):
   
    img_arr = pickle.loads(bin_data.data)

    height = img_arr.shape[0]
    width = img_arr.shape[1]
    
    if len(img_arr.shape) == 3:
            for i in range(height//2):
                img_arr[i], img_arr[height - i - 1] = img_arr[height - i - 1], img_arr[i]
    else:
        for i in range(height//2):
            img_arr[i], img_arr[height - i - 1] = img_arr[height - i - 1], img_arr[i]
    
    pimg = pickle.dumps(img_arr)       
    return xmlrpc.client.Binary(pimg)
server.register_function(send_back_flipped, 'send_back_flipped')

print("Listening on port 8008...")
server.serve_forever()



Listening on port 8008...


127.0.0.1 - - [27/Dec/2023 18:39:18] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2023 18:39:20] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2023 18:39:22] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2023 18:39:24] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2023 18:39:27] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2023 18:39:29] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2023 18:39:31] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2023 18:39:33] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2023 18:39:35] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2023 18:39:37] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2023 18:39:39] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2023 18:39:41] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2023 18:39:43] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2023 18:39:45] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2023 18:39:47] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/202

KeyboardInterrupt: 